In [130]:
import numpy as np
import pandas as pd
import pickle
import re
import string

In [2]:
with open('scotus_tweets_01_31.pkl', 'r') as picklefile: 
    tweets = pickle.load(picklefile)

In [3]:
status = tweets[0]

In [42]:
tweets_clean = [{'tweet_id': tweet.id,
                 'text': tweet.text,
                 'tweet_time': tweet.created_at,
                 'symbols': tweet.entities['symbols'],
                 'mentions': tweet.entities['user_mentions'],
                 'hashtags': tweet.entities['hashtags'],
                 'urls': tweet.entities['urls'],
                 'retweeted': tweet.retweeted,'user_id': tweet.user.id,
                 'user_name': tweet.user.screen_name,
                 'name': tweet.user.name,
                 'bio': tweet.user.description,
                 'followers': tweet.user.followers_count,
                 'following': tweet.user.friends_count,
                 'geo': tweet.geo,
                 'location': tweet.user.location,
                 'source': tweet.source,
                 'verified': tweet.user.verified
                } for tweet in tweets[:10]]

In [45]:
tweets_df = pd.DataFrame(tweets_clean)

In [60]:
tweets_df.mentions[7][0]['screen_name']

u'Harvard_Law'

In [52]:
for txt in tweets_df.text:
    print txt.replace('\n', ' ')

RT @TEN_GOP: Dems lost the White House, Senate, House and 33 states in open elections, yet somehow feel they should be choosing a #SCOTUS j…
#SCOTUS ... Gorsuch !
"I am a man of my word"  lol ok, 45   #SCOTUS
Trump picks Gorsuch for #SCOTUS  https://t.co/KnVPEYL3kH
RT @MillerRoad: I missed the call-in numbers for #scotus #supremecourt
.@realDonaldTrump pays his debt to conservatives w/his selection of Neil Gorsuch! #SCOTUS
BREAKING: Trump selects Judge Neil Gorsuch as his #SCOTUS pick
RT @Harvard_Law: What is originalism? @CassSunstein via @BV https://t.co/vrRRDLlwhH #SCOTUS
It's Gorsuch! #SCOTUSNominee #SCOTUS
RT @emzanotti: You guys, #SCOTUS is the missing bobcat and you know it.


In [67]:
tweets_df['user_mentions'] = [[mention['screen_name'] for mention in mentions] for mentions in tweets_df.mentions]
tweets_df['hashtag_list'] = [[tag['text'] for tag in hashtags] for hashtags in tweets_df.hashtags]
tweets_df['url_list'] = [[url['url'] for url in urls] for urls in tweets_df.urls]

In [70]:
tweets_df

,bio,followers,following,geo,hashtags,location,mentions,name,retweeted,source,...,tweet_time,urls,user_id,user_name,verified,mentions_users,user_mentions,hashtag_list,url_list,text_list
0,Music History Metal Politics,1243,2731,None,"[{u'indices': [130, 137], u'text': u'SCOTUS'}]",,"[{u'indices': [3, 11], u'screen_name': u'TEN_G...",Northland Rebel,False,Twitter for Android,...,2017-02-01 01:04:09,[],800561690613977088,northlandrebel,False,[TEN_GOP],[TEN_GOP],[SCOTUS],[],"[RT, @TEN_GOP:, Dems, lost, the, White, House,..."
1,sarcastic grenade throwing Conservative #waar;...,928,917,None,"[{u'indices': [0, 7], u'text': u'SCOTUS'}]","St. Louis, MO",[],William Allen,False,Twitter Web Client,...,2017-02-01 01:04:09,[],1431404118,WhamaaAllen,False,[],[],[SCOTUS],[],"[#SCOTUS, ..., Gorsuch, !]"
2,jonathan cheban blocked me,403,1609,None,"[{u'indices': [38, 45], u'text': u'SCOTUS'}]","Hell's Kitchen, NYC",[],dj horn,False,Twitter for iPhone,...,2017-02-01 01:04:09,[],19366299,thedirtyderek,False,[],[],[SCOTUS],[],"[""I, am, a, man, of, my, word""\n\nlol, ok,, 45..."
3,Managing Editor of @YahooFinance. Making busin...,18181,2430,None,"[{u'indices': [24, 31], u'text': u'SCOTUS'}]","New York, NY",[],Sam Ro,False,TweetDeck,...,2017-02-01 01:04:09,"[{u'url': u'https://t.co/KnVPEYL3kH', u'indice...",239026022,bySamRo,True,[],[],[SCOTUS],[https://t.co/KnVPEYL3kH],"[Trump, picks, Gorsuch, for, #SCOTUS, , https:..."
4,"Lived, loved and worked in the City of spies.",138,296,None,"[{u'indices': [49, 56], u'text': u'scotus'}, {...",,"[{u'indices': [3, 14], u'screen_name': u'Mille...",Zenn,False,Twitter for iPhone,...,2017-02-01 01:04:09,[],631552505,MzZenn,False,[MillerRoad],[MillerRoad],"[scotus, supremecourt]",[],"[RT, @MillerRoad:, I, missed, the, call-in, nu..."
5,#GOP Guy. GW Prof. - @GSPMgwu. Author. As Seen...,22721,2520,None,"[{u'indices': [82, 89], u'text': u'SCOTUS'}]",Florida/D.C.,"[{u'indices': [1, 17], u'screen_name': u'realD...",Ford O'Connell,False,TweetDeck,...,2017-02-01 01:04:08,[],123899931,FordOConnell,False,[realDonaldTrump],[realDonaldTrump],[SCOTUS],[],"[.@realDonaldTrump, pays, his, debt, to, conse..."
6,Video Reporter at @YahooNews w/ @KatieCouric r...,1194,928,None,"[{u'indices': [50, 57], u'text': u'SCOTUS'}]","New York, NY",[],Summer Delaney,False,Twitter Web Client,...,2017-02-01 01:04:08,[],450397102,Summer_Delaney,True,[],[],[SCOTUS],[],"[BREAKING:, Trump, selects, Judge, Neil, Gorsu..."
7,,1,87,None,"[{u'indices': [84, 91], u'text': u'SCOTUS'}]","Brasília, Brasil","[{u'indices': [3, 15], u'screen_name': u'Harva...",Marcus Paulo DaCunha,False,Twitter for iPhone,...,2017-02-01 01:04:08,"[{u'url': u'https://t.co/vrRRDLlwhH', u'indice...",824335233545801731,marcuspcunha,False,"[Harvard_Law, CassSunstein, BV]","[Harvard_Law, CassSunstein, BV]",[SCOTUS],[https://t.co/vrRRDLlwhH],"[RT, @Harvard_Law:, What, is, originalism?, @C..."
8,Bloomberg congressional and political reporter...,1607,1190,None,"[{u'indices': [14, 28], u'text': u'SCOTUSNomin...","Washington, D.C.",[],Laura Litvan,False,Twitter Web Client,...,2017-02-01 01:04:08,[],313545488,LauraLitvan,False,[],[],"[SCOTUSNominee, SCOTUS]",[],"[It's, Gorsuch!, #SCOTUSNominee, #SCOTUS]"
9,Feminist. Historian. I have Fibromyalgia and i...,1764,599,None,"[{u'indices': [25, 32], u'text': u'SCOTUS'}]","Austin, TX","[{u'indices': [3, 13], u'screen_name': u'emzan...",Virginia in Texas,False,TweetDeck,...,2017-02-01 01:04:08,[],188164477,TooTwistedTV,False,[emzanotti],[emzanotti],[SCOTUS],[],"[RT, @emzanotti:, You, guys,, #SCOTUS, is, the..."


In [150]:
def clean_tweet_text (tweet):
    words = []
    #remove urls
    clean = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
    # remove mentions
    clean = re.sub(r'@+\w+', '', clean)
    # iterate through words
    for word in clean.strip().split():
        # remove punctuation and whitespace
        word = word.strip(string.punctuation + string.whitespace)
        # transform to lowercase
        word = word.lower()
        # add word to words list
        words += [word]
    # remove any outstanding blanks
    words = [word for word in words if word != '']
    return words   

In [151]:
clean_tweet_text(tweets_df.text[7])

[u'rt', u'what', u'is', u'originalism', u'via', u'scotus']

In [69]:
tweets_df['text_clean'] = [tweet.strip().split(' ') for tweet in tweets_df.text]

In [125]:
re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweets_df.text[7])

u'RT @Harvard_Law: What is originalism? @CassSunstein via @BV  #SCOTUS'

In [127]:
re.sub(r'@+\w+', '', tweets_df.text[7])

u'RT : What is originalism?  via  https://t.co/vrRRDLlwhH #SCOTUS'

In [129]:
re.sub(r'@+\w+', '', re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweets_df.text[7])).lower()

u'rt : what is originalism?  via   #scotus'

In [73]:
tweets_df.text[7]

u'RT @Harvard_Law: What is originalism? @CassSunstein via @BV https://t.co/vrRRDLlwhH #SCOTUS'